In [1]:
import re
import os 
import advertools as adv
import pandas as pd
import pyarrow.parquet as pq
pd.options.display.max_columns = None

import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, State, callback
from dash.dash_table import DataTable
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
load_figure_template(['flatly'])
import plotly, jupyter_dash, dash, pyarrow, dash_bootstrap_templates as dbt

for pkg in [adv, pd, pyarrow, plotly, jupyter_dash, dash, dbc, dbt]:
    print(f'{pkg.__name__:-<30}v{pkg.__version__}')

advertools--------------------v0.13.2
pandas------------------------v1.5.0
pyarrow-----------------------v9.0.0
plotly------------------------v5.10.0
jupyter_dash------------------v0.4.2
dash--------------------------v2.6.2
dash_bootstrap_components-----v1.2.1
dash_bootstrap_templates------v1.0.7


## Data processing

In [2]:
def jl_to_parquet(jl_filepath, parquet_filepath):
    status = 'not done'
    crawldf = pd.read_json(jl_filepath, lines=True)
    while status == 'not done':
        try:
            crawldf.to_parquet(parquet_filepath, index=False, version='2.6')
            status = 'done'
        except Exception as e:
            error = e.args[-1]
            column = re.findall('column (\S+)', error)
            print(column[0])
            crawldf[column[0]] = crawldf[column[0]].astype(str).replace('nan', pd.NA)


def parquet_col_types(parquet_filepath, numeric_cols=True):
    """Get a list of column names and their data types from a parquet file.

    `numeric_cols` determines whether you want numeric or non-numeric columns.
    """
    pq_dataset = pq.ParquetDataset(parquet_filepath, use_legacy_dataset=False)
    col_types = dict(zip(pq_dataset.schema.names, pq_dataset.schema.types))
    numeric = {k: v for k, v in col_types.items() if str(v).startswith('int') or str(v).startswith('double')}
    if numeric_cols:
        return numeric
    non_numeric = {k: v for k, v in col_types.items() if not str(v).startswith('int') and not str(v).startswith('double')}
    return non_numeric

## Data visualization

In [3]:
def make_column_hist(df, column):
    fig = px.histogram(
        df,
        x=column,
        title=f'Histogram of <b>{column}</b>')
    fig.data[0].hovertemplate = f'<b>{column.replace("_", " ").title()}</b><br><br>' + fig.data[0].hovertemplate.replace('=', ': ')
    return fig

def make_ecdf(df, column):
    df['count below'] = range(len(df))
    df['count above'] = range(len(df)-1, -1, -1)
    df['total count'] = len(df)
    fig = px.ecdf(
        df.dropna(subset=[column]),
        x=column,
        hover_data=['count below', 'count above', 'total count'],
        title=f'Cumulative distribution of <b>{column}</b>',
        ecdfnorm='percent')
    fig.data[0].hovertemplate = f'<b>{column.replace("_", " ").title()}</b><br><br>{column}' + ': %{x}<br>percent: %{y}<br><br>count below: %{customdata[0]:,}<br>count above: %{customdata[1]:,}<br>total count: %{customdata[2]:,}<extra></extra>'
    fig.layout.yaxis.ticksuffix = '%'
    fig.layout.yaxis.showspikes = True
    fig.layout.xaxis.showspikes = True
    return fig

def make_numeric_card(df, column):
    card = dbc.Col(dbc.Card([
        dbc.CardHeader(f"Summary of {column.replace('_', ' ').title()}"),
        dbc.CardBody([
            html.H5(f"Number of elements: {len(df[column]):,}"),
            html.H5(f"Missing elements: {df[column].isna().sum():,} ({df[column].isna().mean():.1%})"), 
            html.H5(f"Min: {df[column].min():,.1f} | Median: {df[column].median():,.1f} | Max: {df[column].max():,.1f}"),
        ])
        ]), lg=5)
    return card

def make_text_card(df, column):
    num_elements = len(df[column].astype(str).str.split('@@').explode())
    uniques = df[column].astype(str).str.split('@@').explode().nunique()
    card = dbc.Col(dbc.Card([
        dbc.CardHeader(f"Summary of {column.replace('_', ' ').title()}"),
        dbc.CardBody([
            html.H5(f"Number of elements: {num_elements:,}"),
            html.H5(f"Unique elements: {uniques:,}"),
            html.H5(f"Number of pages: {len(df[column]):,}"),
            html.H5(f"Elements per page: {num_elements / len(df[column]):.2f}"),
            html.H5(f"Missing elements: {df[column].isna().sum():,} ({df[column].isna().mean():.1%})"), 
            ])
        ]), lg=5)
    return card


def make_text_counts_table(df, column):
    val_counts = df[column].astype(str).str.split('@@').explode().value_counts().reset_index()
    val_counts.columns = [column, 'count']
    val_counts['count'] = [format(x, ',') for x in val_counts['count']]
    table = DataTable(
        val_counts.to_dict('records'),
        [{"name": i, "id": i} for i in val_counts.columns],
        virtualization=True,
        page_size=350,
        fixed_rows={'headers': True},
        export_format='csv',
        style_header={
            'fontFamily': 'Arial',
            'fontColor': '#2F3B4C',
            'fontWeight': 'bold'},
        style_data={'fontFamily': 'Arial'},
        style_cell={
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'minWidth': 100,
            'maxWidth': 400},
        sort_action='native'
    )
    return table


def make_text_col_hist(df, column):
    count_na = df[column].isna().sum()
    mean_na = df[column].isna().mean()
    count_per_url = [0 if pd.isna(x) else len(str(x).split("@@")) for x in df[column]]
    countdf = pd.DataFrame({
        'count per page': count_per_url,
        'zero': [x == 0 for x in count_per_url]
    })
    counts_hist = px.histogram(
        countdf,
        x='count per page',
        color='zero',
        title=f"Number of <b>{column.replace('_', ' ').title()}</b> elements per URL")
    counts_hist.layout.showlegend = False
    counts_hist.layout.xaxis.title = 'Number per URL'
    counts_hist.layout.modebar = None
    for i, data in enumerate(counts_hist.data):
        data.hovertemplate = re.sub('=', ': ', counts_hist.data[i].hovertemplate)
        data.hovertemplate = re.sub('zero.*?<br>', '', counts_hist.data[i].hovertemplate)
    return counts_hist

In [4]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])


app.layout = html.Div([
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            html.Br(),
            html.H1([html.Code('advertools'),  ' SEO Crawler – Analytics']), html.Br(),
            html.Div([
                dcc.Upload(dbc.Button('Select file'), id='upload_button')
            ], style={'textAlign': 'center'}),
            html.H1('Numeric columns overview'), html.Br(),
            dbc.Col([
                dcc.Dropdown(id='numeric_col_dropdown'),
            ], lg=5),
            dcc.Loading(html.Div(id='numeric_col_summary'))
        ], lg=10)
    ]), html.Br(), html.Br(),
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            html.H1('Text columns overview'), html.Br(),
            dbc.Col([
                dcc.Dropdown(id='text_col_dropdown')
            ], lg=5),
            dcc.Loading(html.Div(id='text_col_summary'))
        ], lg=10)
    ])
] + [html.Br()] * 20)


@app.callback(
    Output('numeric_col_dropdown', 'options'),
    Input('upload_button', 'filename'))
def set_numeric_col_dropdown_options(filename):
    if not filename:
        raise PreventUpdate
    filepath = os.path.abspath(filename)
    numeric_columns = sorted(parquet_col_types(filepath))
    return numeric_columns


@app.callback(
    Output('numeric_col_summary', 'children'),
    Input('numeric_col_dropdown', 'value'),
    Input('upload_button', 'filename'))
def display_numeric_col_summary(column, filename):
    if not column:
        raise PreventUpdate
    filepath = os.path.abspath(filename)
    df = pd.read_parquet(filepath, columns=[column])
    hist = make_column_hist(df, column)
    ecdf = make_ecdf(df, column)
    numeric_card = make_numeric_card(df, column)

    report = html.Div([
        html.Br(), 
        html.Div([
            dcc.Loading([
                numeric_card, html.Br(),
                dbc.Row([
                    dbc.Col([
                        html.Div(dcc.Graph(figure=hist, config={'displayModeBar': False}))
                    ]),
                    dbc.Col([
                        html.Div(dcc.Graph(figure=ecdf, config={'displayModeBar': False}))
                        ])
                    ])
                ])
            ])
        ])
    return report


@app.callback(
    Output('text_col_dropdown', 'options'),
    Input('upload_button', 'filename'))
def set_text_col_dropdown_options(filename):
    if not filename:
        raise PreventUpdate
    filepath = os.path.abspath(filename)
    text_columns = sorted(parquet_col_types(filepath, numeric_cols=False))
    return text_columns


@app.callback(
    Output('text_col_summary', 'children'),
    Input('text_col_dropdown', 'value'),
    Input('upload_button', 'filename'))
def display_text_col_summary(column, filename):
    if not column:
        raise PreventUpdate
    filepath = os.path.abspath(filename)
    df = pd.read_parquet(filepath, columns=[column])
    hist = make_text_col_hist(df, column)
    counts_table = make_text_counts_table(df, column)
    text_card = make_text_card(df, column)

    report = html.Div([
        html.Br(), 
        html.Div([
            dcc.Loading([
                text_card, html.Br(),
                dbc.Row([
                    dbc.Col([
                        html.Div(dcc.Graph(figure=hist, config={'displayModeBar': False}))
                    ]),
                    dbc.Col([
                        html.Br(),
                        html.H5(f"Counts of {column}:"), html.Br(),
                        html.Div(counts_table)
                    ])
                ])
            ])
        ])
    ])
    return report

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
